In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf

import tensorflow_decision_forests as tfdf
print(f"Found TF-DF {tfdf.__version__}")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

Found TF-DF 1.4.0


In [3]:
train_dataframe = pd.read_csv("/kaggle/input/titanic/train.csv")
serving_dataframe = pd.read_csv("/kaggle/input/titanic/test.csv")
train_dataframe.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [5]:
def preprocess(dataframe):
    dataframe = dataframe.copy()
    
    def normalize_name(x):
        return " ".join([v.strip(",()[].\"'") for v in x.split(" ")])
    
    def ticket_number(x):
        return x.split(" ")[-1]
    def ticket_item(x):
        items = x.split(" ")
        if len(items) == 1:
            return "NONE"
        return "_".join(items[0:-1])
    
    dataframe["Name"] = dataframe["Name"].apply(normalize_name)
    dataframe["Ticket_number"] = dataframe["Ticket"].apply(ticket_number)
    dataframe["Ticket_item"] = dataframe["Ticket"].apply(ticket_item)
    return dataframe

preprocessed_train_dataframe = preprocess(train_dataframe)
preprocessed_serving_dataframe = preprocess(serving_dataframe)
preprocessed_train_dataframe.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket_number,Ticket_item
0,1,0,3,Braund Mr Owen Harris,male,22.0,1,0,A/5 21171,7.2500,NaN,S,21171,A/5
1,2,1,1,Cumings Mrs John Bradley Florence Briggs Thayer,female,38.0,1,0,PC 17599,71.2833,C85,C,17599,PC
2,3,1,3,Heikkinen Miss Laina,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,3101282,STON/O2.
3,4,1,1,Futrelle Mrs Jacques Heath Lily May Peel,female,35.0,1,0,113803,53.1000,C123,S,113803,NONE
4,5,0,3,Allen Mr William Henry,male,35.0,0,0,373450,8.0500,NaN,S,373450,NONE


In [6]:
input_features = list(preprocessed_train_dataframe.columns)
input_features.remove("Ticket")
input_features.remove("PassengerId")
input_features.remove("Survived")
print(f"Input Features: {input_features}")

Input Features: ['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'Ticket_number', 'Ticket_item']


In [10]:
def tokenize_names(features, labels=None):
    """Divite the names into tokens. TF-DF can consume text tokens natively."""
    features["Name"] =  tf.strings.split(features["Name"])
    return features, labels

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_train_dataframe,label="Survived").map(tokenize_names)
serving_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_serving_dataframe).map(tokenize_names)

In [12]:
model = tfdf.keras.GradientBoostedTreesModel(verbose=0,
                                            features = [tfdf.keras.FeatureUsage(name=n) for n in input_features],
                                            exclude_non_specified_features=True,
                                            random_seed=1234,
                                            )
model.fit(train_ds)

self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss:{self_evaluation.loss}")

[WARNING 23-08-11 14:10:39.1366 UTC gradient_boosted_trees.cc:1818] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-08-11 14:10:39.1368 UTC gradient_boosted_trees.cc:1829] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-08-11 14:10:39.1369 UTC gradient_boosted_trees.cc:1843] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 23-08-11 14:10:47.7309 UTC kernel.cc:1243] Loading model from path /tmp/tmpd88aetsl/model/ with prefix 52debf42926d4af4
[INFO 23-08-11 14:10:47.7425 UTC abstract_model.cc:1311] Engine "GradientBoostedTreesQuickScorerExtended" built
[INFO 23-08-11 14:10:47.7427 UTC kernel.cc:1075] Use fast generic engine


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Accuracy: 0.8260869383811951 Loss:0.8608942627906799


In [19]:
model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0,
    features = [tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features = True,
    min_examples = 1,
    categorical_algorithm = "RANDOM",
    shrinkage = 0.05,
    sparse_oblique_normalization = "MIN_MAX",
    sparse_oblique_num_projections_exponent = 1.5,
    split_axis = "SPARSE_OBLIQUE",
    random_seed = 1234,
    num_trees = 2000,
)

model.fit(train_ds)

self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss:{self_evaluation.loss}")

[WARNING 23-08-11 14:18:32.1961 UTC gradient_boosted_trees.cc:1818] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-08-11 14:18:32.1962 UTC gradient_boosted_trees.cc:1829] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-08-11 14:18:32.1962 UTC gradient_boosted_trees.cc:1843] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 23-08-11 14:18:33.2067 UTC kernel.cc:1243] Loading model from path /tmp/tmpbjpym367/model/ with prefix 6d53c289c6c244c6
[INFO 23-08-11 14:18:33.2187 UTC decision_forest.cc:660] Model loaded with 47 root(s), 2519 node(s), and 10 input feature(s).
[INFO 23-08-11 14:18:33.2188 UTC abstract_model.cc:1311] Engine "GradientBoostedTreesGeneric" built
[INFO 23-08-11 14:18:33.2188 UTC kernel.cc:1075] Use fast generic engine


Accuracy: 0.77173912525177 Loss:0.9877318739891052


In [20]:
model.summary()

Model: "gradient_boosted_trees_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (11):
	Age
	Cabin
	Embarked
	Fare
	Name
	Parch
	Pclass
	Sex
	SibSp
	Ticket_item
	Ticket_number

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.           "Sex"  0.645527 ################
    2.           "Age"  0.346852 #####
    3.          "Fare"  0.256332 ##
    4.          "Name"  0.206208 #
    5. "Ticket_number"  0.183464 
    6.   "Ticket_item"  0.181181 
    7.      "Embarked"  0.180923 
    8.         "Parch"  0.179493 
    9.        "Pclass"  0.179280 
   10.         "SibSp"  0.171551 

Variable Importance: NUM_AS_ROOT:
    1.  "Sex" 41.000000 ################
    2. "Name"  5.000000 #
  

In [23]:
def prediction_to_kaggle_format(model, threshold = 0.5):
    proba_survive = model.predict(serving_ds, verbose = 0)[:,0]
    return pd.DataFrame({
        "PassengerId": serving_dataframe["PassengerId"],
        "Survived": (proba_survive ==threshold).astype(int)
    })

def make_submission(kaggle_predictions):
    path = "/kaggle/working/submission.csv"
    kaggle_predictions.to_csv(path, index = False)
    print(f"Submission exported to {path}")
    
kaggle_predictions = prediction_to_kaggle_format(model)
make_submission(kaggle_predictions)
!head /kaggle/working/submission.csv

Submission exported to /kaggle/working/submission.csv
PassengerId,Survived
892,0
893,0
894,0
895,0
896,0
897,0
898,0
899,0
900,0


In [27]:
tuner = tfdf.tuner.RandomSearch(num_trials = 1000)
tuner.choice("min_examples", [2, 5, 7, 10])
tuner.choice("categorical_algorithm", ["CART", "RANDOM"])

local_search_space = tuner.choice("growing_strategy", ["LOCAL"])
local_search_space.choice("max_depth", [3, 4, 5, 6, 8])
global_search_space = tuner.choice("growing_strategy", ["BEST_FIRST_GLOBAL"], merge = True)
global_search_space.choice("max_num_nodes", [16, 32, 64, 128, 256])

tuner.choice("shrinkage", [0.02, 0.05, 0.10, 0.15])
tuner.choice("num_candidate_attributes_ratio", [0.2, 0.5, 0.9, 1.0])
tuner.choice("split_axis", ["AXIS_ALIGNED"])
oblique_space = tuner.choice("split_axis", ["SPARSE_OBLIQUE"], merge = True)
oblique_space.choice("sparse_oblique_normalization",
                    ["NONE", "STANDARD_DEVIATION", "MIN_MAX"])
oblique_space.choice("sparse_oblique_weights", ["BINARY", "CONTINUOUS"])
oblique_space.choice("sparse_oblique_num_projections_exponent", [1.0, 1.5])

tuned_model = tfdf.keras.GradientBoostedTreesModel(tuner = tuner)
tuned_model.fit(train_ds, verbose = 0)

tuned_self_evaluation = tuned_model.make_inspector().evaluation()
print(f"Accuracy: {tuned_self_evaluation.accuracy} Loss:{tuned_self_evaluation.loss}")

Use /tmp/tmpnsiheb7r as temporary training directory


[WARNING 23-08-11 16:00:18.2782 UTC gradient_boosted_trees.cc:1818] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-08-11 16:00:18.2782 UTC gradient_boosted_trees.cc:1829] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-08-11 16:00:18.2782 UTC gradient_boosted_trees.cc:1843] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 23-08-11 16:02:19.9266 UTC kernel.cc:1243] Loading model from path /tmp/tmpnsiheb7r/model/ with prefix 60abbfb62147436d
[INFO 23-08-11 16:02:19.9426 UTC decision_forest.cc:660] Model loaded with 32 root(s), 852 node(s), and 12 input feature(s).
[INFO 23-08-11 16:02:19.9426 UTC abstract_model.cc:1311] Engine "GradientBoostedTreesGeneric" built
[INFO 23-08-11 16:02:19.9427 UTC kernel.cc:1075] Use fast generic engine


Accuracy: 0.8630136847496033 Loss:0.6804219484329224


In [38]:
predictions = None
num_predictions = 0

for i in range(100):
    print(f"i:{i}")
    model = tfdf.keras.GradientBoostedTreesModel(
        verbose = 0,
        features = [tfdf.keras.FeatureUsage(name = n) for n in input_features],
        exclude_non_specified_features = True,
        random_seed = i,
        honest = True,
    )
    
    model.fit(train_ds)

    sub_predictions = model.predict(serving_ds, verbose = 0) [:,0]
    if predictions is None:
        predictions = sub_predictions
    else:
        predictions += sub_predictions
    num_predictions += 1
    
predictions/=num_predictions
kaggle_predictions = pd.DataFrame({
        "PassengerId": serving_dataframe["PassengerId"],
        "Survived": (predictions >= 0.5).astype(int)
    })
    
make_submission(kaggle_predictions)
#special thanks to Gusthema, a fellow Kaggle user and contributor

i:0


[WARNING 23-08-11 16:25:40.0647 UTC gradient_boosted_trees.cc:1818] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-08-11 16:25:40.0647 UTC gradient_boosted_trees.cc:1829] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-08-11 16:25:40.0647 UTC gradient_boosted_trees.cc:1843] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 23-08-11 16:25:40.7794 UTC kernel.cc:1243] Loading model from path /tmp/tmplnbfr90u/model/ with prefix a4426d27a0a44781
[INFO 23-08-11 16:25:40.7840 UTC abstract_model.cc:1311] Engine "GradientBoostedTreesQuickScorerExtended" built
[INFO 23-08-11 16:25:40.7840 UTC kernel.cc:1075] Use fast generic engine


i:1


[WARNING 23-08-11 16:25:41.2895 UTC gradient_boosted_trees.cc:1818] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-08-11 16:25:41.2896 UTC gradient_boosted_trees.cc:1829] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-08-11 16:25:41.2896 UTC gradient_boosted_trees.cc:1843] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 23-08-11 16:25:42.3888 UTC kernel.cc:1243] Loading model from path /tmp/tmpve7exfsz/model/ with prefix 3b7c7a3851284350
[INFO 23-08-11 16:25:42.4086 UTC kernel.cc:1075] Use fast generic engine


i:2


...

i:98


[WARNING 23-08-11 16:28:08.2330 UTC gradient_boosted_trees.cc:1818] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-08-11 16:28:08.2330 UTC gradient_boosted_trees.cc:1829] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-08-11 16:28:08.2331 UTC gradient_boosted_trees.cc:1843] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 23-08-11 16:28:09.0822 UTC kernel.cc:1243] Loading model from path /tmp/tmp8np8a88v/model/ with prefix dc598db0db9c42e3
[INFO 23-08-11 16:28:09.0903 UTC kernel.cc:1075] Use fast generic engine


i:99


[WARNING 23-08-11 16:28:09.5556 UTC gradient_boosted_trees.cc:1818] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-08-11 16:28:09.5556 UTC gradient_boosted_trees.cc:1829] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-08-11 16:28:09.5556 UTC gradient_boosted_trees.cc:1843] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 23-08-11 16:28:10.5911 UTC kernel.cc:1243] Loading model from path /tmp/tmpfxhq9of1/model/ with prefix 992fd96a60134b74
[INFO 23-08-11 16:28:10.6064 UTC kernel.cc:1075] Use fast generic engine


Submission exported to /kaggle/working/submission.csv
